In [1]:
import vocab
import re

import torch
import torch.nn as nn
import torch.nn.functional as F

/Users/outbell/Ajay/DeepLearning/GenAI/smallGPT/.venv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


In [22]:
t = torch.randint(0, 100, (2, 2, 2))
print(t[0, -1, -1])
if t[0, -1, -1] in torch.tensor([77, 78]):
    print("yes")

tensor(78)
yes


In [2]:
torch.randint(0,0, (1, ))

RuntimeError: random_ expects 'from' to be less than 'to', but got from=0 >= to=0